In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing.data import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import preprocessing


columns = ['dataset','building','has_act','start','end'] + range(54)
data = pd.read_csv('windat_kettle.csv', names=columns, index_col=None, 
                   header=None, parse_dates=['start','end'], date_parser=pd.to_datetime)
data.head()

,dataset,building,has_act,start,end,0,1,2,3,4,...,44,45,46,47,48,49,50,51,52,53
0,ukdale,1,1,2013-03-17 20:40:27,2013-03-17 20:53:27,640.849976,646.190002,650.030029,642.590027,638.859985,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN
1,ukdale,1,1,2013-03-17 20:40:33,2013-03-17 20:53:33,646.190002,650.030029,642.590027,638.859985,642.559998,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN
2,ukdale,1,1,2013-03-17 20:40:59,2013-03-17 20:53:59,646.190002,650.030029,642.590027,638.859985,642.559998,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN
3,ukdale,1,1,2013-03-17 20:41:25,2013-03-17 20:54:25,650.030029,642.590027,638.859985,642.559998,639.109985,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN
4,ukdale,1,1,2013-03-17 20:41:27,2013-03-17 20:54:27,650.030029,642.590027,638.859985,642.559998,639.109985,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN


In [8]:
# separar por dataset
data_redd = data[data['dataset'] == 'redd']
data_ukdale = data[data['dataset'] == 'ukdale']

# separar por si tienen o no activaciones
data_redd_act = data_redd[data_redd['has_act'] == 1]
data_redd_noact = data_redd[data_redd['has_act'] == 0]
data_ukdale_act = data_ukdale[data_ukdale['has_act'] == 1]
data_ukdale_noact = data_ukdale[data_ukdale['has_act'] == 0]

print "Casas de REDD con activaciones: " + str(data_redd_act['building'].unique())
print "Casas de REDD sin activaciones: " + str(data_redd_noact['building'].unique())
print "Casas de UK-DALE con activaciones: "+ str(data_ukdale_act['building'].unique())
print "Casas de UK-DALE sin activaciones: " + str(data_ukdale_noact['building'].unique())

Casas de REDD con activaciones: []
Casas de REDD sin activaciones: []
Casas de UK-DALE con activaciones: [1]
Casas de UK-DALE sin activaciones: [1 2 3 4 5]


No es posible separar una casa para validar

In [9]:
data_train = data_ukdale

In [10]:
data_train = data_train[range(54)]
data_train = data_train.fillna(0)

d = np.shape(data_train)[1]/2
X_train = preprocessing.scale(np.array(data_train.iloc[:,:d]), with_mean=False, with_std=True)
y_train = preprocessing.scale(np.array(data_train.iloc[:,d:]), with_mean=False, with_std=True)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

In [11]:
regressor = MLPRegressor(solver='adam',
                        alpha=1e-5,
                        activation = 'tanh',
                        verbose = False,
                        hidden_layer_sizes=(500,500,500,),
                        tol=1e-8,
                        learning_rate='adaptive',
                        learning_rate_init=0.001,
                        random_state=1,
                        early_stopping=False,
                        validation_fraction=0.2)

regressor = regressor.fit(X_train, y_train)

In [12]:
print "Train data: score= {}".format(regressor.score(X_train, y_train))
print "Test data: score= {}".format(regressor.score(X_test, y_test))

Train data: score= 0.924267015692
Test data: score= 0.783280099541
